In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI：Vertex AI 迁移：AutoML 文本实体提取

<table align="left">
  <td>
<a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-text-classification-batch-prediction.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-text-classification-batch-prediction.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
         <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-automl-text-classification-batch-prediction.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>   
</table>
<br/><br/><br/>

## 概览

本笔记本演示了如何使用Vertex AI ncbi疾病研究数据集创建一个AutoML文本实体提取模型，以及如何为批量预测提供模型。您需要提供一个存储数据集的存储桶。

注意：您可能需要支付与测试此SDK相关的培训、预测、存储或其他Google Cloud产品使用费用。

了解更多关于[迁移至Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai)和[文本数据的实体提取](https://cloud.google.com/vertex-ai/docs/training-overview#entity_extraction_for_text)。

### 目标

这个笔记本的目标是构建一个自动机器学习文本实体提取模型。已经采取了以下步骤：
此教程使用以下谷歌云机器学习服务：

* Vertex AI 数据集资源
* AutoML 训练
* Vertex AI 模型资源
* Vertex AI 批量预测

执行的步骤包括以下内容：

* 设置任务名称和 GCS 前缀
* 复制 AutoML 视频演示培训数据以创建托管数据集
* 在 Vertex AI 上创建数据集
* 配置训练作业
* 启动训练作业，并在 Vertex AI 上创建模型
* 复制 AutoML 视频演示预测数据，以创建批量预测作业
* 在模型上执行批量预测作业

### 数据集

本教程使用的数据集是来自[国家生物技术信息中心](https://www.ncbi.nlm.nih.gov/)的[NCBI疾病研究摘要数据集](https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/)。您将在本教程中使用的数据集版本存储在公共云存储桶中。

成本

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

安装

安装执行该笔记本所需的软件包。

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 cuda-python \
                                 google-cloud-storage
if os.getenv("IS_TESTING"):
    ! pip3 install --upgrade --quiet tensorflow 

只适用于Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在你开始之前

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面： [找到项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改Vertex AI使用的`REGION`变量。了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行实况教程会话，可能会使用一个共享的测试账户或项目。为了避免在创建的资源之间发生命名冲突，您可以为每个实例会话创建一个 UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经验证过了。

**2. 本地 JupyterLab 实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 与人合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务帐户或其他
* 查看如何授予云存储权限给您的服务帐户，请访问https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

## 初始化用于Python的Vertex AI SDK

为您的项目和对应的存储桶初始化Python的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

云存储训练数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中JSONL索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/language/ucaip_ten_dataset.jsonl"

快速查看您的数据

本教程使用存储在公共云存储桶中的NCBI生物医学数据集的一个版本，使用一个JSONL索引文件。

首先快速查看数据。通过计算JSONL索引文件中对象的数量（`wc -l`）来计数示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

创建一个数据集

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### 创建数据集

接下来，使用`TextDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟。

In [ ]:
dataset = aip.TextDataset.create(
    display_name="NCBI Biomedical" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.text.extraction,
)

print(dataset.resource_name)

示例输出：

    信息：google.cloud.aiplatform.datasets.dataset：创建TextDataset
    信息：google.cloud.aiplatform.datasets.dataset：创建TextDataset支持的LRO：projects/759209241365/locations/us-central1/datasets/3704325042721521664/operations/3193181053544038400
    信息：google.cloud.aiplatform.datasets.dataset：已创建TextDataset。资源名称：projects/759209241365/locations/us-central1/datasets/3704325042721521664
    信息：google.cloud.aiplatform.datasets.dataset：要在另一个会话中使用此TextDataset：
    信息：google.cloud.aiplatform.datasets.dataset：ds = aiplatform.TextDataset('projects/759209241365/locations/us-central1/datasets/3704325042721521664')
    信息：google.cloud.aiplatform.datasets.dataset：正在导入TextDataset数据：projects/759209241365/locations/us-central1/datasets/3704325042721521664
    信息：google.cloud.aiplatform.datasets.dataset：导入TextDataset数据支持的LRO：projects/759209241365/locations/us-central1/datasets/3704325042721521664/operations/5152246891450204160
    信息：google.cloud.aiplatform.datasets.dataset：TextDataset数据已导入。资源名称：projects/759209241365/locations/us-central1/datasets/3704325042721521664
    projects/759209241365/locations/us-central1/datasets/3704325042721521664

训练一个模型

### [训练.automl-api](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练管道，2) 运行该管道。

#### 创建训练管道

一个AutoML训练管道是通过`AutoMLTextTrainingJob`类创建的，具有以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：模型训练的任务类型。
  - `classification`：文本分类模型。
  - `sentiment`：文本情感分析模型。
  - `extraction`：文本实体提取模型。
- `multi_label`：如果是分类任务，单标签（False）还是多标签（True）。
- `sentiment_max`：如果是情感分析任务，则是情感值的最大值。

实例化的对象是训练管道的有向无环图（DAG）。

In [ ]:
dag = aip.AutoMLTextTrainingJob(
    display_name="biomedical_" + UUID, prediction_type="extraction"
)

print(dag)

示例输出：

<google.cloud.aiplatform.training_jobs.AutoMLTextTrainingJob object at 0x7fc3b6c90f10>

#### 运行训练流水线

接下来，通过调用方法 `run`，运行DAG来启动训练作业，参数如下:

- `dataset`：要训练模型的 `Dataset` 资源。
- `model_display_name`：已训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留存数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。

完成 `run` 方法将返回 `Model` 资源。

训练流水线的执行将耗时最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="biomedical_" + UUID,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

示例输出：

    信息：google.cloud.aiplatform.training_jobs：查看训练：
    https://console.cloud.google.com/ai/platform/locations/us-central1/training/8859754745456230400?project=759209241365
    信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob 项目/759209241365/位置/us-central1/trainingPipelines/8859754745456230400 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob 项目/759209241365/位置/us-central1/trainingPipelines/8859754745456230400 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob 项目/759209241365/位置/us-central1/trainingPipelines/8859754745456230400 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob 项目/759209241365/位置/us-central1/trainingPipelines/8859754745456230400 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob 项目/759209241365/位置/us-central1/trainingPipelines/8859754745456230400 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    ...
    信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob 运行完成。 资源名称：项目/759209241365/位置/us-central1/trainingPipelines/8859754745456230400
    信息：google.cloud.aiplatform.training_jobs：模型可在项目/759209241365/位置/us-central1/models/6389525951797002240 获取。

评估模型

### [projects.locations.models.evaluations.list](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

### [projects.locations.models.evaluations.list](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

## 检查模型评估分数
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用在部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=biomedical_" + UUID)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

示例输出：

    名称："projects/759209241365/locations/us-central1/models/623915674158235648/evaluations/4280507618583117824"
    指标模式URI："gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml"
    指标{
      struct_value {
        字段 {
          键: "auPrc"
          值 {
            数值: 0.9891107
          }
        }
        字段 {
          键: "confidenceMetrics"
          值 {
            列表值 {
              值 {
                struct_value {
                  字段 {
                    键: "precision"
                    值 {
                      数值: 0.2
                    }
                  }
                  字段 {
                    键: "recall"
                    值 {
                      数值: 1.0
                    }
                  }
                }
              }

## 进行批量预测

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

制作测试项

您将使用合成数据作为测试数据项。不要担心我们使用合成数据 - 我们只是想演示如何进行预测。

In [ ]:
test_item_1 = 'Molecular basis of hexosaminidase A deficiency and pseudodeficiency in the Berks County Pennsylvania Dutch.\tFollowing the birth of two infants with Tay-Sachs disease ( TSD ) , a non-Jewish , Pennsylvania Dutch kindred was screened for TSD carriers using the biochemical assay . A high frequency of individuals who appeared to be TSD heterozygotes was detected ( Kelly et al . , 1975 ) . Clinical and biochemical evidence suggested that the increased carrier frequency was due to at least two altered alleles for the hexosaminidase A alpha-subunit . We now report two mutant alleles in this Pennsylvania Dutch kindred , and one polymorphism . One allele , reported originally in a French TSD patient ( Akli et al . , 1991 ) , is a GT-- > AT transition at the donor splice-site of intron 9 . The second , a C-- > T transition at nucleotide 739 ( Arg247Trp ) , has been shown by Triggs-Raine et al . ( 1992 ) to be a clinically benign " pseudodeficient " allele associated with reduced enzyme activity against artificial substrate . Finally , a polymorphism [ G-- > A ( 759 ) ] , which leaves valine at codon 253 unchanged , is described'
test_item_2 = "Analysis of alkaptonuria (AKU) mutations and polymorphisms reveals that the CCC sequence motif is a mutational hot spot in the homogentisate 1,2 dioxygenase gene (HGO).	We recently showed that alkaptonuria ( AKU ) is caused by loss-of-function mutations in the homogentisate 1 , 2 dioxygenase gene ( HGO ) . Herein we describe haplotype and mutational analyses of HGO in seven new AKU pedigrees . These analyses identified two novel single-nucleotide polymorphisms ( INV4 + 31A-- > G and INV11 + 18A-- > G ) and six novel AKU mutations ( INV1-1G-- > A , W60G , Y62C , A122D , P230T , and D291E ) , which further illustrates the remarkable allelic heterogeneity found in AKU . Reexamination of all 29 mutations and polymorphisms thus far described in HGO shows that these nucleotide changes are not randomly distributed ; the CCC sequence motif and its inverted complement , GGG , are preferentially mutated . These analyses also demonstrated that the nucleotide substitutions in HGO do not involve CpG dinucleotides , which illustrates important differences between HGO and other genes for the occurrence of mutation at specific short-sequence motifs . Because the CCC sequence motifs comprise a significant proportion ( 34 . 5 % ) of all mutated bases that have been observed in HGO , we conclude that the CCC triplet is a mutational hot spot in HGO ."

### 制作批量输入文件

现在制作一个批量输入文件，您将把它存储在您的本地云存储桶中。批量输入文件只能是JSONL格式。对于JSONL文件，您需要为每个数据项（实例）在一行中制作一个字典条目。该字典包含键/值对：

- `content`：文本项文件的云存储路径。
- `mime_type`：内容类型。在我们的示例中，是一个`text`文件。

例如：

                        {'content': '[your-bucket]/file1.txt', 'mime_type': 'text'}

In [ ]:
import json

import tensorflow as tf

gcs_test_item_1 = BUCKET_URI + "/test1.txt"
with tf.io.gfile.GFile(gcs_test_item_1, "w") as f:
    f.write(test_item_1 + "\n")
gcs_test_item_2 = BUCKET_URI + "/test2.txt"
with tf.io.gfile.GFile(gcs_test_item_2, "w") as f:
    f.write(test_item_2 + "\n")

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": gcs_test_item_1, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")
    data = {"content": gcs_test_item_2, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

进行批量预测请求

现在您的模型资源已经训练好，您可以通过调用batch_predict()方法进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批处理请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `sync`：如果设置为True，则调用将在等待异步批处理作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="biomedical_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

示例输出：

    信息：google.cloud.aiplatform.jobs：正在创建BatchPredictionJob
    <google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0> 正在等待上游依赖项完成。
    信息：google.cloud.aiplatform.jobs：已创建BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
    信息：google.cloud.aiplatform.jobs：要在另一个会话中使用此BatchPredictionJob：
    信息：google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
    信息：google.cloud.aiplatform.jobs：查看批量预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296 当前状态：
    JobState.JOB_STATE_RUNNING

等待批处理预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻止批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

*示例输出:*

    INFO: google.cloud.aiplatform.jobs:创建了BatchPredictionJob。资源名称: projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    INFO: google.cloud.aiplatform.jobs:在另一个会话中使用此BatchPredictionJob：
    INFO: google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
    INFO: google.cloud.aiplatform.jobs:查看批量预测作业:
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    INFO: google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态:
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态:
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态:
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态:
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态:
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态:
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态:
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态:
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态:
    JobState.JOB_STATE_SUCCEEDED
    INFO: google.cloud.aiplatform.jobs:BatchPredictionJob运行已完成。资源名称: projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的 Cloud Storage 输出存储桶中。您可以调用方法 iter_outputs() 来获取生成结果的每个 Cloud Storage 文件的列表。每个文件以 JSON 格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `ids`：每个预测请求的内部分配的唯一标识符。
 - `displayNames`：每个类别标签的类名。
 - `confidences`：每个类别标签的预测置信度，介于 0 到 1 之间。
 - `textSegmentStartOffsets`：实体在文本中起始的字符偏移量。
 - `textSegmentEndOffsets`：实体在文本中结束的字符偏移量。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

{'instance': {'content': 'gs://andy-1234-221921aip-20210811180202/test2.txt', 'mimeType': 'text/plain'}, 'prediction': {'ids': ['2208238262504390656', '2208238262504390656', '4827081445820334080', '4827081445820334080', '2208238262504390656', '4827081445820334080', '4827081445820334080'], 'displayNames': ['具体疾病', '具体疾病', '修饰语', '修饰语', '具体疾病', '修饰语', '修饰语'], 'textSegmentStartOffsets': ['208', '193', '381', '522', '670', '26', '12'], 'textSegmentEndOffsets': ['210', '204', '383', '524', '672', '28', '23'], 'confidences': [0.99951637, 0.9994987, 0.9994574, 0.9994488, 0.99924797, 0.9969406, 0.9692179]}}

做在线预测

### [predictions.deploy-model-api](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

### [predictions.deploy-model-api](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

部署模型

接下来，部署您的模型进行在线预测。要部署模型，您需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

*示例输出：*

    信息：google.cloud.aiplatform.models:创建端点
    信息：google.cloud.aiplatform.models:创建端点支持的 LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
    信息：google.cloud.aiplatform.models:端点已创建。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    信息：google.cloud.aiplatform.models:在另一个会话中使用此端点：
    信息：google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
    信息：google.cloud.aiplatform.models:将模型部署到端点：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    信息：google.cloud.aiplatform.models:部署端点模型支持的 LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
    信息：google.cloud.aiplatform.models:端点模型已部署。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472

### [predictions.online-prediction-automl](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

### [在线预测自动ML](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

制作测试项目

您将使用合成数据作为测试数据项目。不要担心我们使用的是合成数据 - 我们只是想演示如何进行预测。

In [ ]:
test_item = 'Molecular basis of hexosaminidase A deficiency and pseudodeficiency in the Berks County Pennsylvania Dutch.\tFollowing the birth of two infants with Tay-Sachs disease ( TSD ) , a non-Jewish , Pennsylvania Dutch kindred was screened for TSD carriers using the biochemical assay . A high frequency of individuals who appeared to be TSD heterozygotes was detected ( Kelly et al . , 1975 ) . Clinical and biochemical evidence suggested that the increased carrier frequency was due to at least two altered alleles for the hexosaminidase A alpha-subunit . We now report two mutant alleles in this Pennsylvania Dutch kindred , and one polymorphism . One allele , reported originally in a French TSD patient ( Akli et al . , 1991 ) , is a GT-- > AT transition at the donor splice-site of intron 9 . The second , a C-- > T transition at nucleotide 739 ( Arg247Trp ) , has been shown by Triggs-Raine et al . ( 1992 ) to be a clinically benign " pseudodeficient " allele associated with reduced enzyme activity against artificial substrate . Finally , a polymorphism [ G-- > A ( 759 ) ] , which leaves valine at codon 253 unchanged , is described'

### 进行预测

现在您的`Model`资源已部署到一个`Endpoint`资源，您可以通过向`Endpoint`资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为：

     { 'content': text_string }

由于predict()方法可以获取多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

predict()调用的响应是一个带有以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `displayNames`：每个实体的类名。
- `confidences`：每个实体的预测置信度，介于0和1之间。
- `textSegmentStartOffsets`：文本中实体的起始偏移量。
- `textSegmentEndOffsets`：文本中实体的结束偏移量。
- `deployed_model_id`：执行预测的已部署`Model`资源的Vertex AI标识符。

In [ ]:
instances_list = [{"content": test_item}]

prediction = endpoint.predict(instances_list)
print(prediction)

*示例输出：*

预测（predictions=[{'displayNames': ['具体疾病', '具体疾病', '具体疾病', '修饰语', '修饰语', '修饰语'], 'confidences': [0.9995822906494141, 0.999564528465271, 0.9995641708374023, 0.9993661046028137, 0.9993420839309692, 0.9993830323219299], 'textSegmentStartOffsets': [19.0, 148.0, 168.0, 235.0, 329.0, 687.0], 'textSegmentEndOffsets': [46.0, 165.0, 171.0, 238.0, 332.0, 690.0], 'ids': ['1746900775675625472', '1746900775675625472', '1746900775675625472', '8664429803316707328', '8664429803316707328', '8664429803316707328']}], deployed_model_id='7103029833386426368', explanations=None)

解除部署模型

当您完成预测时，您可以从`Endpoint`资源取消部署模型。这将取消所有计算资源，并结束已部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理工作

要清理本教程中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Delete the dataset using the Vertex dataset object

dataset.delete()

# Delete the model using the Vertex model object

model.delete()

# Delete the endpoint using the Vertex endpoint object

endpoint.delete()
# Delete the AutoML or Pipeline trainig job

dag.delete()

# Delete the batch prediction job using the Vertex batch prediction object

batch_predict_job.delete()

# Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object

# Delete GCS bucket.
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI